In [1]:
import csv

import numpy as np
import pandas as pd

from bokeh.plotting import output_notebook, show, figure

from sklearn.metrics import classification_report, accuracy_score
from sklearn.cross_validation import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Activation

output_notebook()

Using Theano backend.


Loading BokehJS ...

In [2]:
label_colors = {0: 'green', 1: 'red'}

In [3]:
features_by_axis = [
    'radius',
    'texture',
    'perimeter',
    'area',
    'smoothness',
    'compactness',
    'concavity',
    'concave points',
    'symmetry',
    'fractal_dimension',
]

columns = ['case', 'type']
input_columns = []
for axis in 'xyz':
    for feature in features_by_axis:
        feature_name = '{}_{}'.format(axis, feature)
        columns.append(feature_name)
        input_columns.append(feature_name)

label_column = 'type'

In [4]:
data = pd.read_csv('wdbc.data', names=columns)
data['type'] = data['type'].map(lambda x: 1 if x == 'M' else 0)
print(data.shape)

(569, 32)


In [5]:
data.head()

,case,type,x_radius,x_texture,x_perimeter,x_area,x_smoothness,x_compactness,x_concavity,x_concave points,...,z_radius,z_texture,z_perimeter,z_area,z_smoothness,z_compactness,z_concavity,z_concave points,z_symmetry,z_fractal_dimension
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [6]:
feature1 = 'x_area'
feature2 = 'y_area'

colors = data[label_column].map(lambda x: label_colors[x])

f = figure()

f.xaxis.axis_label = "feature " + str(feature1)
f.yaxis.axis_label = "feature " + str(feature2)

f.circle(data[feature1], 
         data[feature2], 
         color=colors)
show(f)

In [7]:
train_data, test_data =  train_test_split(data, train_size=0.8)

In [8]:
model = Sequential([
    Dense(100, input_dim=len(input_columns)),
    Activation('tanh'),
    Dense(100),
    Activation('tanh'),
    Dense(1),
    Activation('sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)

model.fit(
    train_data[input_columns].values, 
    train_data[label_column].values, 
    nb_epoch=100,
    batch_size=32,
)

Epoch 1/100
455/455 [==============================] - 0s - loss: 0.4390 - acc: 0.7890     
Epoch 2/100
455/455 [==============================] - 0s - loss: 0.2511 - acc: 0.9143     
Epoch 3/100
455/455 [==============================] - 0s - loss: 0.2351 - acc: 0.9231     
Epoch 4/100
455/455 [==============================] - 0s - loss: 0.2347 - acc: 0.9165     
Epoch 5/100
455/455 [==============================] - 0s - loss: 0.2052 - acc: 0.9253     
Epoch 6/100
455/455 [==============================] - 0s - loss: 0.2142 - acc: 0.9275     
Epoch 7/100
455/455 [==============================] - 0s - loss: 0.1951 - acc: 0.9275     
Epoch 8/100
455/455 [==============================] - 0s - loss: 0.2164 - acc: 0.9187     
Epoch 9/100
455/455 [==============================] - 0s - loss: 0.2201 - acc: 0.9209     
Epoch 10/100
455/455 [==============================] - 0s - loss: 0.2010 - acc: 0.9319     
Epoch 11/100
455/455 [==============================] - 0s - loss: 0.2101 - acc

In [9]:
train_predictions = model.predict(train_data[input_columns].values)
test_predictions = model.predict(test_data[input_columns].values)

sets = (
    ('train', train_data[label_column].values, train_predictions),
    ('test', test_data[label_column].values, test_predictions),
)

for name, labels, predictions in sets:
    predictions_as_int = np.rint(predictions)
    print('#' * 50)
    print(name)
    print('accuracy', accuracy_score(labels, predictions_as_int))
    print(classification_report(labels, predictions_as_int))
    print('predictions', dict(pd.DataFrame(predictions_as_int)[0].value_counts().items()))
    print()


##################################################
train
accuracy 0.934065934066
             precision    recall  f1-score   support

          0       0.94      0.95      0.95       286
          1       0.92      0.90      0.91       169

avg / total       0.93      0.93      0.93       455

predictions {0.0: 290, 1.0: 165}

##################################################
test
accuracy 0.877192982456
             precision    recall  f1-score   support

          0       0.93      0.87      0.90        71
          1       0.81      0.88      0.84        43

avg / total       0.88      0.88      0.88       114

predictions {0.0: 67, 1.0: 47}

